In [ ]:
%cd /content/
! git clone https://github.com/joeynmt/joeynmt.git

In [ ]:
% cd joeynmt/
!pip3 install .

In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#preprocessing the data into subword tokens
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files

#learn bpe on the training data
#! cat vocab.$src
! subword-nmt learn-joint-bpe-and-vocab --input train.en train.ro -s 30000 -o bpe.codes.30000 --write-vocabulary vocab.en vocab.ro

#apply BPE splits to the dev and test data
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < train.en > train.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < train.ro > train.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < val.en > val.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < val.ro > val.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < test.en > test.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < test.ro > test.bpe.ro

/content/drive/MyDrive/data_en_ro/dcep_training_files


In [ ]:
! pip install -U sacremoses

! sacremoses -l "en" -j 8 tokenize < train.en > tok_train.en
! sacremoses -l "ro" -j 8 tokenize < train.ro > tok_train.ro

! sacremoses -l "en" -j 8 tokenize < val.en > tok_val.en
! sacremoses -l "ro" -j 8 tokenize < val.ro > tok_val.ro

! sacremoses -l "en" -j 8 tokenize < test.en > tok_test.en
! sacremoses -l "ro" -j 8 tokenize < test.ro > tok_test.ro


Requirement already up-to-date: sacremoses in /usr/local/lib/python3.7/dist-packages (0.0.45)
100% 783896/783896 [02:43<00:00, 4805.20it/s]
100% 783896/783896 [02:04<00:00, 6292.90it/s]
100% 2000/2000 [00:03<00:00, 608.97it/s]
100% 2000/2000 [00:02<00:00, 768.00it/s]
100% 2000/2000 [00:03<00:00, 617.02it/s]
100% 2000/2000 [00:02<00:00, 844.84it/s]


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_tokenization/

! subword-nmt learn-joint-bpe-and-vocab --input tok_train.en tok_train.ro -s 30000 -o bpe.codes.30000 --write-vocabulary vocab.en vocab.ro

#apply BPE splits to the dev and test data
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < tok_train.en > tok_train.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < tok_train.ro > tok_train.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < tok_val.en > tok_val.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < tok_val.ro > tok_val.bpe.ro

! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.en < tok_test.en > tok_test.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 --vocabulary vocab.ro < tok_test.ro > tok_test.bpe.ro

/content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_tokenization


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/
!wc -l *

/content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE
    30001 bpe.codes.30000
     2000 test.bpe.en
     2000 test.bpe.ro
   783896 train.bpe.en
   783896 train.bpe.ro
     2000 val.bpe.en
     2000 val.bpe.ro
    21712 vocab.en
    30206 vocab.ro
  1657711 total


In [ ]:
!head /content/drive/MyDrive/data_en_ro/dcep_full/clean_vocab.txt

,
&quot;
the
de
of
a
and
.
to
în


In [ ]:
!wc -l /content/drive/MyDrive/data_en_ro/dcep_full/*.bpe.*

    10000 /content/drive/MyDrive/data_en_ro/dcep_full/clean_dev.bpe.en
    10000 /content/drive/MyDrive/data_en_ro/dcep_full/clean_dev.bpe.ro
   157580 /content/drive/MyDrive/data_en_ro/dcep_full/clean_test.bpe.en
   157580 /content/drive/MyDrive/data_en_ro/dcep_full/clean_test.bpe.ro
   620316 /content/drive/MyDrive/data_en_ro/dcep_full/clean_train.bpe.en
   620316 /content/drive/MyDrive/data_en_ro/dcep_full/clean_train.bpe.ro
  1575792 total


In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE

/content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE


In [ ]:
! sudo chmod 777 /content/joeynmt/scripts/build_vocab.py
! /content/joeynmt/scripts/build_vocab.py train.bpe.en train.bpe.ro --output_path vocab.txt


#some output
! echo "BPE Sentences english"
! head -n 5 /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/test.bpe.en

! echo "BPE Sentences ro"
! head -n 5 /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/test.bpe.ro

! echo "Combined BPE Vocab"
! head -n 10 /content/drive/MyDrive/data_en_ro/dcep_training_files/splitted_BPE/vocab.txt

BPE Sentences english
Considers the Agency to be a source of important environmental information for all EU institutions and policy-@@ mak@@ ing; notes with satisfaction that the Agency has been able to trans@@ form some complic@@ ated data into clear conclusions and to communicate them to the public, such as the Agency's environmental statement 2006 and specific reports on bi@@ o energy and greenhouse gas emission trends and pro@@ jec@@ tions in Europe;
00@@ 02@@ /2009 )
Urges all the institutions to cooperate construc@@ tively with the Ombudsman at all stages of the procedure, to ab@@ ide by am@@ ic@@ able sett@@ lement@@ s, to follow up his critical remarks and to apply his draft recommend@@ ations;
having regard to its resolution of 14 December 2006 on A European strategy for sustainable, competitive and secure energy − Green paper Texts adopted, P6_TA(2006)0@@ 60@@ 3 .
These pap@@ ers began to be submitted to ACP parliaments at the end of 2007 and it is important for the Committee

In [ ]:
%cd /content/drive/MyDrive/data_en_ro/dcep_full/

/content/drive/MyDrive/data_en_ro/dcep_full


In [ ]:
!wc -l /home/bernadeta/BA_code/data/00-raw/EN-RO-bisentences.txt

25055 clean_vocab.txt


In [33]:
from bs4 import BeautifulSoup

with open("/home/bernadeta/BA_code/data/newstest2016-enro-ref.ro.sgm") as new1_ro, open("newstest_ro_strip.txt", "w") as new_ro:
    soup=BeautifulSoup(new1_ro, 'html.parser')
    new_ro.write(soup.get_text())

with open("/home/bernadeta/BA_code/data/newstest2016-enro-src.en.sgm") as new1_en, open("newstest_en_strip.txt", "w") as new_en:
    soup=BeautifulSoup(new1_en, 'html.parser')
    new_en.write(soup.get_text())

In [36]:
import pandas as pd

df_ro=pd.read_csv("newstest_ro_strip.txt", sep="\n")
df_ro.to_csv("newstest_ro.txt", index=False)

df_en=pd.read_csv("newstest_en_strip.txt", sep="\n+", skip_blank_lines=True)
df_en.to_csv("newstest_en.txt", index=False)


In [74]:
newstest_both=pd.concat([df_en, df_ro], axis=1)

In [75]:
pd.set_option('display.max_colwidth', None)
newstest_both

,UN Chief Says There Is No Military Solution in Syria,Șeful ONU declară că nu există soluții militare în Siria
0,"Secretary-General Ban Ki-moon says his response to Russia's stepped up military support for Syria is that ""there is no military solution"" to the nearly five-year conflict and more weapons will only worsen the violence and misery for millions of people.",Secretarul General Ban Ki-moon afirmă că răspunsul său la suportul militar al Rusiei pentru Siria este că „nu există o soluție militară” la conflictul care durează de aproape cinci ani iar mai multe arme nu ar face decât să agraveze violența și suferința a milioane de oameni.
1,"The U.N. chief again urged all parties, including the divided U.N. Security Council, to unite and support inclusive negotiations to find a political solution.","Șeful ONU a solicitat din nou tuturor părților, inclusiv Consiliului de securitate ONU divizat să se unifice și să susțină negocierile pentru a găsi o soluție politică."
2,"Ban told a news conference Wednesday that he plans to meet with foreign ministers of the five permanent council nations - the U.S., Russia, China, Britain and France - on the sidelines of the General Assembly's ministerial session later this month to discuss Syria.","Ban a declarat miercuri în cadrul unei conferințe că intenționează să se întâlnească luna aceasta cu miniștrii de externe din cinci țări permanent prezente în consiliu - SUA, Rusia, China, Anglia și Franța - pe marginea sesiunii ministeriale a Adunării Generale pentru a discuta despre Siria."
3,"He expressed regret that divisions in the council and among the Syrian people and regional powers ""made this situation unsolvable.""",Ban și-a exprimat regretul că divizările în consiliu și între poporul sirian și puterile regionale „au făcut această situație de nerezolvat”.
4,Ban urged the five permanent members to show the solidarity and unity they did in achieving an Iran nuclear deal in addressing the Syria crisis.,"Ban le-a cerut celor cinci membri permanenți să dea dovadă de solidaritatea și unitatea arătate atunci când au reușit să încheie un acord referitor la armele nucleare ale Iranului, abordând astfel criza din Siria."
...,...,...
1993,Money is enough.,Nu sunt bani puțini.
1994,Sometimes I'm ashamed to take my money from the cashier's office.,Uneori mi-e rușine să ridic banii de la casierie.
1995,At the end of the office I will report everything I spent besides the cabinet employees' salaries and its maintenance.,"La sfârșitul mandatului voi face un raport cu tot ce am cheltuit, în afara salariilor angajaților de la cabinet și întreținerea acestuia."
1996,"One month I happened to collect 30,000 lei with the reimbursements for transport, accommodation and other expenses.","S-a întâmplat să ridic într-o lună și 30.000 de lei, cu toate deconturile de transport, cazare și celelalte."


## Preprocess newstest2016 to try trained models on it

In [1]:
! pip install -U sacremoses

! sacremoses -l "en" -j 8 tokenize < /home/bernadeta/BA_code/data/newstest_en.txt > tok_newstest.en
! sacremoses -l "ro" -j 8 tokenize < /home/bernadeta/BA_code/data/newstest_ro.txt > tok_newstest.ro

100%|█████████████████████████████████████| 1999/1999 [00:00<00:00, 2383.91it/s]


In [2]:
%cd /home/bernadeta/BA_code/data/02-preprocessed/02-02-bicleaner-preprocessed

/home/bernadeta/BA_code/data/02-preprocessed/02-02-bicleaner-preprocessed


In [4]:
! subword-nmt apply-bpe -c bpe.codes.30000 < /home/bernadeta/BA_code/data/tok_newstest.en > tok_newstest.bpe.en
! subword-nmt apply-bpe -c bpe.codes.30000 < /home/bernadeta/BA_code/data/tok_newstest.ro > tok_newstest.bpe.ro

In [5]:
!wc -l tok_newstest*

  1999 tok_newstest.bpe.en
  1999 tok_newstest.bpe.ro
  3998 insgesamt


In [6]:
!head tok_newstest*

==> tok_newstest.bpe.en <==
UN Chief S@@ ays There Is No Mili@@ tary S@@ olution in Syria
&quot; Secretary-General Ban Ki-@@ mo@@ on says his response to Russia &apos;s stepped up military support for Syria is that &quot; &quot; there is no military solution &quot; &quot; to the nearly five-year conflict and more weapons will only wor@@ sen the violence and mis@@ ery for millions of people . &quot;
&quot; The U@@ .@@ N. chief again urged all parties , including the divided U@@ .@@ N. Security Council , to unite and support inclusive negotiations to find a political solution . &quot;
&quot; Ban told a news conference Wednesday that he plans to meet with foreign ministers of the five permanent counci@@ l nations - the U@@ .@@ S. , Russia , China , Bri@@ tain and France - on the si@@ de@@ lines of the General Assembly &apos;s ministerial session later this month to discuss Syria . &quot;
&quot; He expressed regret that divisions in the counci@@ l and among the Syrian people and regional p